In [ ]:
# check if you have the below libraries if not use "pip install" syntax to install them
# Importing libraries
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('Api-Key.csv') # save your api key in a file of your choice which you access; I saved it in a csv file
rapid_api_key = df.loc[df['API']== 'RapidApi']['KEY'].iloc[0]

In [ ]:
import requests

url = "https://aerodatabox.p.rapidapi.com/flights/airports/iata/YYZ" # here we have used API to get the data of Toronto Pearson Airpot(YYZ), 
                                                                     # you can also try codes from other airport and see what results you get!
                                                                     # type the airport code after "iata/<your airport code>", in short just replace YYZ with desired airport code!

querystring = {"offsetMinutes":"-120","durationMinutes":"720","withLeg":"true","direction":"Both","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
	"x-rapidapi-key": rapid_api_key, # place your api key here; PLEASE KEEP IN MIND API-KEY SHOULD BE ANONYMOUS and private
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
new_response = response.json()
print(response.status_code, '\n') 
print(new_response)

In [ ]:
departures = new_response.get('departures', [])
df = pd.json_normalize(departures)
    
# Cleaning up column names (making it more readable)
df.columns = df.columns.str.replace('.', '_')
df

In [ ]:
print('--Data Info--','\n')
df.info()    

In [ ]:
print('--Data Shape--','\n')
print(f" {df.shape[0]} rows, {df.shape[1]} columns")

In [ ]:
for i, col in enumerate(df.columns):
    print(f"{i+1}. {col}")

In [ ]:
df_copy = df.copy()

In [ ]:
missing_info = df_copy.isnull().sum()
missing_info

In [ ]:
missing_df = pd.DataFrame({
        'Column': missing_info.index,
        'Missing_Count': missing_info.values,
    }).sort_values('Missing_Count', ascending=False)
    
print("Missing values by column:")
print(missing_df[missing_df['Missing_Count'] > 0])

In [ ]:
# Dropping columns with higher percentage of missing values

threshold = 0.9 # here 0.9 indicates 90%, meaning those columns that have 90% of their data missing
missing_percent = df.isnull().sum() / len(df)
columns = missing_percent[missing_percent > threshold].index.tolist()

print(f"Dropping {len(columns)} columns with >{threshold*100}% missing values:")
for col in columns:
    print(f"  {col}: {missing_pct[col]:.1%} missing")

# creating another dataset which will be further used for exploratory analysis
df_clean = df.drop(columns = columns)

#Remaining columns
print(f"\nRemaining columns: {df_clean.shape[1]}")
print(f"Remaining rows: {df_clean.shape[0]}")

remaining_missing = df_clean.isnull().sum()
print("\nMissing values in remaining columns:")
print(remaining_missing[remaining_missing > 0].sort_values(ascending=False))

In [ ]:
# Converting datetime columns
datetime_columns = [col for col in df_clean.columns if 'Time' in col and 'utc' in col.lower()]
for col in datetime_columns:
    if col in df_clean.columns:
        # Removing 'Z' suffix and converting them to datetime
        df_clean[col] = pd.to_datetime(df_clean[col].str.replace('Z', ''), errors='coerce')
        print(f"Converted {col} to datetime") # This print statement acts like a validation

In [ ]:
df_clean

In [ ]:
# Calculating flight duration
if 'departure_scheduledTime_utc' in df_clean.columns and 'arrival_scheduledTime_utc' in df_clean.columns:
    df_clean['scheduled_duration_hours'] = round((
    df_clean['arrival_scheduledTime_utc'] - df_clean['departure_scheduledTime_utc']
    ).dt.total_seconds() / 3600, 2)
    print("Added scheduled_duration_hours column to the dataframe.")
    
# Calculating delays both for arrivals and departures
if 'departure_revisedTime_utc' in df_clean.columns and 'departure_scheduledTime_utc' in df_clean.columns:
    df_clean['departure_delay_minutes'] = abs((
    df_clean['departure_revisedTime_utc'] - df_clean['departure_scheduledTime_utc']
    ).dt.total_seconds() / 60) # using absolute function here to avoid negative values to keep things simple
    print("Added departure_delay_minutes column to the dataframe.")
    
if 'arrival_revisedTime_utc' in df_clean.columns and 'arrival_scheduledTime_utc' in df_clean.columns:
    df_clean['arrival_delay_minutes'] = abs((
    df_clean['arrival_revisedTime_utc'] - df_clean['arrival_scheduledTime_utc']
    ).dt.total_seconds() / 60) # using absolute function here to avoid negative values to keep things simple
    print("Added arrival_delay_minutes column to the dataframe.")
    
# Extracting hour from departure time
if 'departure_scheduledTime_utc' in df_clean.columns:
    df_clean['departure_hour'] = df_clean['departure_scheduledTime_utc'].dt.hour
    print("Added departure_hour column to the dataframe.")

In [ ]:
df_clean

In [ ]:
# removing some more columns
df_clean = df_clean.drop(columns = ['arrival_baggageBelt'], errors = 'ignore')
df_clean = df_clean.drop(columns = ['airline_icao'], errors = 'ignore')

In [ ]:
# --------ANALYSIS--------

print('--Airline Analysis--', '\n')
if 'airline_name' in df_clean.columns:
        # Top airlines by number of flights
        airline_counts = df_clean['airline_name'].value_counts().head(10)
        print("Top 10 Airlines by Number of Flights:")
        print(airline_counts)
        
# Airline performances
if 'departure_delay_minutes' in df_clean.columns:
    airline_performance = df_clean.groupby('airline_name').agg({
    'departure_delay_minutes': ['mean', 'median', 'std', 'count']
    }).round(2)
            
    airline_performance.columns = ['Avg_Delay', 'Median_Delay', 'Std_Delay', 'Flight_Count']
    airline_performance = airline_performance.sort_values('Avg_Delay', ascending=False)
            
    print("\nAirline Delay Performance (Top 10 by flight count):")
    top_airlines = airline_counts.head(10).index
    print(airline_performance.loc[top_airlines])
        

plt.figure(figsize=(12, 6))
airline_counts.plot(kind='bar')
plt.title('Top 10 Airlines by Number of Flights')
plt.xlabel('Airline')
plt.ylabel('Number of Flights')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
print('-- Delay Analysis --', '\n')

delay_columns = ['departure_delay_minutes', 'arrival_delay_minutes']
available_delays = [col for col in delay_columns if col in df_clean.columns]
    
if available_delays:
    for col in available_delays:
        print(f"\n{col.replace('_', ' ').title()} Statistics:")
        delay_stats = df_clean[col].describe()
        print(delay_stats)
            
        # Calculating on-time performance
        on_time = (df_clean[col] <= 15).sum()  # Within 15 minutes is generally considered on-time
        total = df_clean[col].notna().sum()
        on_time_percent = (on_time / total) * 100
        print(f"On-time performances: {on_time_percent:.1f}%")
            
        
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        df_clean[col].hist(bins=50, alpha=0.7)
        plt.title(f'{col.replace("_", " ").title()} Distribution')
        plt.xlabel('Delay (minutes)')
        plt.ylabel('Frequency')
            
        plt.show()

In [ ]:
print("-- Airport Analysis --", '\n')
    
if 'arrival_airport_name' in df_clean.columns:
    # Top destination airports
    airport_counts = df_clean['arrival_airport_name'].value_counts().head(10)
    print("Top 10 Destination Airports:")
    print(airport_counts)
    
    plt.figure(figsize=(12, 6))
    airport_counts.plot(kind='bar')
    plt.title('Top 10 Destination Airports')
    plt.xlabel('Airport')
    plt.ylabel('Number of Flights')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
print('-- Aircraft Model Analysis --')

if 'aircraft_model' in df_clean.columns:
    # Top aircraft models used
    aircraft_counts = df_clean['aircraft_model'].value_counts().head(10)
    print("Top 10 Aircraft Models:")
    print(aircraft_counts)
        
    
    plt.figure(figsize=(12, 6))
    aircraft_counts.plot(kind='bar')
    plt.title('Top 10 Aircraft Models')
    plt.xlabel('Aircraft Model')
    plt.ylabel('Number of Flights')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
print('-- Analysing Hourly Patterns --', '\n')

if 'departure_hour' in df_clean.columns:
    # Departures by hour
    hourly_flights = df_clean['departure_hour'].value_counts().sort_index()
    print("Flights by Hour of Day:")
    print(hourly_flights)
    
    plt.figure(figsize=(12, 6))
    hourly_flights.plot(kind='line')
    plt.title('Flight Departures by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Number of Flights')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

## Alternate Scenario

#### If we only wanted to look at a particular section, say the arrivals section, then our code would have looked like this. Similarly, if we want to look at other sections we would follow the same steps.

In [ ]:
def arrivals_data(data):
    
    # First step, start by getting the departures list (this contains all flights)
    departures = data.get('departures', [])
    
    # Second, extract only the arrival data from each flight by using get function
    arrivals_list = []
    for flight in departures:
        arrival_data = flight.get('arrival', {})
        if arrival_data:  # Only adds arrival data to the list if it exists
            # Adding flight number for reference
            arrival_data['flight_number'] = flight.get('number', '')
            arrivals_list.append(arrival_data)
    
    # Lastly, converting the list to DataFrame using json_normalize
    if arrivals_list:
        df = pd.json_normalize(arrivals_list)
        # Cleaning up column names
        df.columns = df.columns.str.replace('.', '_')
        return df
    else:
        return pd.DataFrame()

arrivals_data(new_response)